# TO GATHER LINKS FROM CAR DEKHO

<b> STEP 1: </b> Setting up chromium browser for colab

In [1]:
!apt-get update
!apt-get install chromium chromium-driver

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:3 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Get:9 http://deb.debian.org/debian buster/main amd64 Packages [10.7 MB]
Hit:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:12 http://deb.debian.org/debian buster-updates/main amd64 Packages [9,745 B]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:14 http://deb.debian.org/debian-security buster/updates/main amd64 Packag

In [2]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.nlZ933B62J/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.an0icqXit4/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.3vHR2jaVHE/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

__STEP 2:__ Importing all the required packages

In [3]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.options import Options

In [5]:
def get_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [14]:
def get_car_divs(driver):
    driver.maximize_window()
    driver.get('https://www.cardekho.com/used-cars+in+hyderabad')
    time.sleep(2)

    for i in range(8000):
        driver.execute_script("scrollBy(0,200)")
        time.sleep(1)

    car_divs = driver.find_elements(By.CLASS_NAME, 'NewUcExCard')
    return car_divs

In [7]:
def car_links(car_divs):
    links = []

    for div in car_divs:
        link = div.find_element(By.TAG_NAME, 'a')
        links.append(link.get_attribute('href'))
        
    return links

In [8]:
def get_car_links():
    links = car_links(car_divs)
    return links

__STEP 3:__ Gathering all the links from the main function

In [15]:
if __name__ == '__main__':
    driver = get_driver()
    car_divs = get_car_divs(driver)

    links = get_car_links()

    driver.close()

In [23]:
len(links)

1560

'https://www.cardekho.com/used-car-details/used-Ford-Freestyle-Titanium-Petrol-Bsiv-cars-Hyderabad_90cb1cb4-e6a9-4b63-b742-b3d402c15d3f.htm'

__STEP 4:__ Saving all the links in Google Drive in the form of "links.txt"

In [17]:
from google.colab import drive
drive.mount("mnt")

Mounted at mnt


In [18]:
%cd "/content/mnt/MyDrive/Scrapper"

/content/mnt/MyDrive/Scrapper


In [24]:
with open('hyderabad_links.txt','w') as f:
    for link in links:
        f.write(link + '\n')